## Imports

In [6]:
from toyencoder import CKKSToyEncoder
from encoder import CKKSEncoder
from scheme import CkksScheme
from numpy.polynomial import Polynomial
import numpy as np
from sympy import nextprime

## Try the toy encoder homomorphism

In [7]:
ckks_encoder = CKKSToyEncoder(8)       # So the maximum size vector is 4 (8/2)
poly1 = ckks_encoder.sigma_inverse([2+3j, 3-9j, 4-2j, 1+4j])
poly2 = ckks_encoder.sigma_inverse([1-1j, 5+5j, 6-6j, 9-7j])

print("Poly1: ", poly1)
print("Poly2: ", poly2)

poly_multi = poly1 * poly2
poly_sum = poly1 + poly2

print("Poly_Multi: ", poly_multi)
print("Poly_Sum: ", poly_sum)

real_multi = ckks_encoder.sigma(poly_multi)
real_sum = ckks_encoder.sigma(poly_sum)

print("Real_Multi: ", real_multi)
print("Real_Sum: ", real_sum)

Poly1:  (2.500000000000002-1.0000000000000007j) +
(0.7071067811865489+3.1819805153394625j)·x -
(1.499999999999999-0.4999999999999988j)·x² +
(2.1213203435596393-3.181980515339467j)·x³
Poly2:  (5.249999999999997-2.2500000000000013j) -
(3.181980515339465+2.8284271247461894j)·x +
(1.2499999999999976-1.7500000000000004j)·x² -
(2.8284271247461907+0.35355339059326985j)·x³
Poly_Multi:  (10.875000000000004-10.875000000000009j) +
(0.08838834764831383+11.225320151336422j)·x +
(1.374999999999992-11.750000000000014j)·x² +
(9.192388155425071-14.142135623730955j)·x³ -
(17.625000000000014+1.8749999999999751j)·x⁴ +
(1.5026019100213932-8.573669721886883j)·x⁵ -
(7.1249999999999805-8.25000000000002j)·x⁶
Poly_Sum:  (7.75-3.2500000000000018j) - (2.4748737341529163-0.35355339059327306j)·x -
(0.25000000000000133+1.2500000000000018j)·x² -
(0.7071067811865515+3.535533905932737j)·x³
Real_Multi:  [ 5. +1.j 60.-30.j 12.-36.j 37.+29.j]
Real_Sum:  [ 3.+2.j  8.-4.j 10.-8.j 10.-3.j]


## Try the CKKS encoder

In [8]:
ckks_encoder = CKKSEncoder(8, scale=2**30)       # So the maximum size vector is 4 (8/2)
c_vec1 = [2, 3]
c_vec2 = [1, 5]

poly1 = ckks_encoder.encode(c_vec1)
poly2 = ckks_encoder.encode(c_vec2)

print("Poly1: ", poly1)
print("Poly2: ", poly2)

poly_multi = poly1 * poly2 % ckks_encoder.module
poly_sum = poly1 + poly2

print("Poly_Multi: ", poly_multi)
print("Poly_Sum: ", poly_sum)

real_multi = ckks_encoder.decode(poly_multi/ckks_encoder.scale)
real_sum = ckks_encoder.decode(poly_sum)

print("Real_Multi: ", real_multi)
print("Real_Sum: ", real_sum)

Poly1:  2.68435456e+09 - (3.79625062e+08)·x - 1.0·x² + (3.79625062e+08)·x³
Poly2:  3.22122547e+09 - (1.51850025e+09)·x + 1.0·x² + (1.51850025e+09)·x³
Poly_Multi:  9.79983279e+18 - (5.29905099e+18)·x - (1.57245824e+08)·x² +
(5.29905099e+18)·x³
Poly_Sum:  5.90558003e+09 - (1.89812531e+09)·x + 0.0·x² + (1.89812531e+09)·x³
Real_Multi:  [ 2.+3.85764753e-10j 15.+1.12990506e-10j]
Real_Sum:  [3.+6.58544108e-10j 8.+6.58545440e-10j]


### Encryption and Decryption

In [11]:
def print_vector(vector):
    for i in vector:
        print("-> ", i)

# Parameters
M = 8                       # Degree of the cyclotomic ring (must be power of 2)
p = 2**20                   # Primo usado no contexto de reliniarização (exemplo)
scale = 2**20               # Scale factor for encoding (decimal precision of 10 bits)
noise_scale = 0.1           # Scale of the noise added during encryption
qs = [2**20, 2**40, 2**60]  # Different moduli for the ciphertexts


# Instantiate the encoder
encoder = CKKSEncoder(M=M, scale=scale)

# Generate a random secret key
secret_key_coeffs = np.random.randint(0, 2, encoder.N)
secret_key = Polynomial(secret_key_coeffs)

# Create CKKS scheme instance
scheme = CkksScheme(encoder, secret_key, noise_scale, p=p, qs=qs)
z1 = np.array([1, 8])
z2 = np.array([2, 2])

# Encryption
c1 = scheme.encrypt(z1)
c2 = scheme.encrypt(z2)

c_add = scheme.add(c1, c2)
c_multi = scheme.mult(c1, c2)

# Decryption
z_add = scheme.decrypt(c_add)
z_multi = scheme.decrypt(c_multi)

print("Soma:", z1+z2)
print_vector(z_add)
print("Polynomial")
print_vector(c_add)



print("\nMulti: ", z1*z2)
print_vector(z_multi)
print("Polynomial")
print_vector(c_multi)

Soma: [ 3 10]
->  (3.000001920114222+2.0101482329781817e-06j)
->  (9.999999481787324-2.1253755511096983e-07j)
Polynomial
->  6815745.29885766 - 2595092.14054099·x + 0.53675314·x² +
2595092.6471779·x³ - 0.01096185·x⁴ - 0.58249937·x⁵ - 1.02910372·x⁶ -
0.2471054·x⁷ - 0.24708394·x⁸ + 0.02596854·x⁹
->  -0.79584582 - 0.27860924·x - 0.67603129·x² + 0.73359261·x³ +
(2.14543281e-05)·x⁴ + 0.27305249·x⁵ - 0.02596854·x⁶

Multi:  [ 2 16]
->  (1.9999989132444904+1.6280238916976941e-06j)
->  (15.99999917940688-2.793247366739138e-07j)
Polynomial
->  9437191.0 - 5190188.0·x - 1.0·x² + 5190187.0·x³ + 3.0·x⁴ - 1.0·x⁵ -
6.0·x⁶ + -0.0·x⁷ - 2.0·x⁸ - 1.0·x⁹ + 1.0·x¹⁰ + 0.0·x¹¹ + 0.0·x¹² +
0.0·x¹³ + -0.0·x¹⁴ + -0.0·x¹⁵ + -0.0·x¹⁶ + -0.0·x¹⁷ + -0.0·x¹⁸
->  -3.0 + 3.0·x - 3.0·x² + 2.0·x³ - 1.0·x⁴ + 1.0·x⁵ + 1.0·x⁶ - 1.0·x⁷ +
0.0·x⁸ + -0.0·x⁹ + -0.0·x¹⁰ + 0.0·x¹¹ + 0.0·x¹² + 0.0·x¹³ + 0.0·x¹⁴ +
0.0·x¹⁵
